In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2023-11-11 19:54:44.973082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 19:54:44.973115: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 19:54:44.973149: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os
import shutil

# Source directory containing images
src_dir = './TRAIN'

# Destination directory to copy images
dst_dir = './TEST'

# Check if destination directory exists, create it if it doesn't
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Loop through all files in source directory
for file_name in os.listdir(src_dir):
    # Check if file ends with .jpg
    if len(os.listdir(dst_dir)) == 50:
        break
    if file_name.endswith('.jpg'):
        # Construct full file paths
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        # Copy file to destination directory
        shutil.copy(src_path, dst_path)




In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the YOLO model
yolo_model = YOLO('./best.pt').to(device)
# Load the ResNet50 model
model2 = load_model('./model_resnet50.h5')

# Class mapping
class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './TEST'

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('set.csv')

# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

# Define the target size for ResNet50
target_size = (256, 256)
preds=[]
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)

            # Predict the class of the cropped object using the loaded ResNet50 model
            img_array = np.array(resized_cropped_im)
            img_array = tf.image.resize(img_array, (256, 256))
            img_array = np.expand_dims(img_array, axis=0)
            predictions = model2.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class_label = list(class_mapping.keys())[predicted_class_index[0]]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor(predicted_class_index).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([predictions.max()]).to(device)
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([(class_mapping[filtered_df.iloc[minind]['class']])]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/image022.jpg: 480x640 4 leafs, 89.9ms
Speed: 2.6ms preprocess, 89.9ms inference, 230.5ms postprocess per image at shape (1, 3, 480, 640)


1/1 [==============================] - 1s 1s/step
5   2
1/1 [==============================] - 0s 20ms/step
5   1
1/1 [==============================] - 0s 19ms/step
5   3
1/1 [==============================] - 0s 16ms/step



image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/green-pepper-plant-diseases-bacterial-spot-on-pepper-plant-green-pepper-plant-diseases-photos.jpg: 480x640 5 leafs, 6.3ms
Speed: 1.8ms preprocess, 6.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)


5   4
1/1 [==============================] - 0s 16ms/step
5   1
1/1 [==============================] - 0s 16ms/step
5   3
1/1 [==============================] - 0s 15ms/step
5   0
1/1 [==============================] - 0s 18ms/step
5   2
1/1 [==============================] - 0s 19ms/step


5   4


...

3   1
1/1 [==============================] - 0s 15ms/step
5   1
1/1 [==============================] - 0s 18ms/step
5   2
1/1 [==============================] - 0s 18ms/step
5   3
1/1 [==============================] - 0s 19ms/step
5   3
1/1 [==============================] - 0s 18ms/step
5   0
1/1 [==============================] - 0s 20ms/step
5   4
1/1 [==============================] - 0s 18ms/step
5   3
1/1 [==============================] - 0s 15ms/step



image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/6242739939180544.jpeg.jpg: 640x480 1 leaf, 7.7ms
Speed: 1.8ms preprocess, 7.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)


5   4
1/1 [==============================] - 0s 17ms/step



image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/powdery-mildew-1.jpg: 448x640 2 leafs, 6.6ms
Speed: 1.6ms preprocess, 6.6ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)


1   0
1/1 [==============================] - 0s 19ms/step
6   5
1/1 [==============================] - 0s 20ms/step
6   3


In [6]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [7]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print(preds[i])
    print(data[i])
metrics.update(preds, data)
print(metrics.compute())

{'boxes': tensor([[100.8345,  35.8086, 173.5686,  82.3703]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.7308], device='cuda:0')}
{'boxes': tensor([[ 94,  33, 174,  82]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[2.2778e+02, 2.2066e-01, 3.4100e+02, 6.8144e+01]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.9428], device='cuda:0')}
{'boxes': tensor([[232,   2, 334,  63]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[197.5414,  49.8298, 252.8160, 131.8664]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9991], device='cuda:0')}
{'boxes': tensor([[193,  57, 260, 143]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[136.1872,  62.6859, 213.2086, 217.6389]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.4730], device='cuda:0')}
{'boxes': tensor([[139,  78, 205,